In [121]:
import numpy as np
import os, os.path
import matplotlib.image as matlib
import json

In [122]:
plane = "axial"
mouse_x = 149
mouse_y = 120
axial_slice = '177'
sagittal_slice = '322'
coronal_slice = '150'

In [123]:
npy_file = "../react-ui/src/assets/P57-16_updated/mri_rotated/indices_" + plane + "/slice_" + str(axial_slice) + ".npy"

In [124]:
npy_slice_array = np.load(npy_file)

print(np.isfortran(npy_slice_array))
print(npy_slice_array.shape)
print(npy_slice_array[300])
#print(npy_slice_array)

False
(450, 226)
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1 44 44 44 44 44 44 44 44 44 44 44
 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44  0  0  0  0
  0 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44
 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44
 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0]


In [125]:
# get the block number from the numpy array
block = npy_slice_array[mouse_y][mouse_x]
print(block)

7


In [126]:
# define a vector to be used later (to get the histology coords)
# an earlier version of this notebook did +1 to the mri_slice here, need to check if this is necessary.
# need to find out the correct order of params when creating the vector (depending on the current plane)
if block == 0:
    print("block number is 0, no further calculations are necessary as there is no associated histology")

    
#if plane == "sagittal":
#  mri_vector=np.array([float(mri_slice), float(mouse_y), float(mouse_x), 1.0])
#elif plane == "coronal":
#  mri_vector=np.array([float(mri_slice), float(mouse_y), float(mouse_x), 1.0])
#elif plane == "axial":
#  mri_vector=np.array([float(mouse_x), float(mouse_y), float(mri_slice), 1.0])

mri_vector = np.array([float(sagittal_slice), float(coronal_slice), float(axial_slice), 1.0])

In [138]:
# open the matrix txt file
matrix_txt = open("../react-ui/src/assets/P57-16_updated/mri_rotated/matrices/block_" + str(block) + ".txt")
read_matrix_txt = matrix_txt.read()

In [139]:
# parse the text file and convert it to an array
matrix = read_matrix_txt.replace('\n', ' ')
matrix = matrix.split(" ")
matrix = matrix[:-1] # pops the last element in the array

In [140]:
# convert the array of strings to an array of floats
matrix_as_floats = []
for item in matrix:
    matrix_as_floats.append(float(item))

In [141]:
# divide matrix array into chuncks to form a multidimensional array
def divide_chunks(l, n):
    for i in range(0, len(l), n): 
        yield l[i:i + n]

# how many elements should have
n = 4
  
chunked_array = list(divide_chunks(matrix_as_floats, n))
print(chunked_array)

[[-0.6539785013261993, -1.2529864276694398, -3.765398774658713, 1256.103850878735], [3.9318829633011614, -0.47062047729014717, -0.5558042211829891, 284.41941706211304], [0.07086043352760864, 1.0005244597366796, -0.37536195122105953, -245.8121326422468], [0.0, 0.0, 0.0, 1.0]]


In [142]:
transform = np.array(chunked_array)
print(transform)

[[-6.53978501e-01 -1.25298643e+00 -3.76539877e+00  1.25610385e+03]
 [ 3.93188296e+00 -4.70620477e-01 -5.55804221e-01  2.84419417e+02]
 [ 7.08604335e-02  1.00052446e+00 -3.75361951e-01 -2.45812133e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [143]:
blockvector = transform.dot(mri_vector)

In [144]:
print(blockvector)

[ 1.91099226e+02  1.38151531e+03 -1.39355469e+02  1.00000000e+00]


In [145]:
# test_transform = np.array([[0.31385434307807203, -3.664457652247729, 1.6199030926177134, 666.0080271821023],
# [-0.7744411837137424, 1.4870493065731998, 3.6535143658756977, -139.31333561581766],
# [-1.2110952755381341, -0.18593341100766062, -0.1914204859080043, 118.27995513465419],
# [0.0, 0.0, 0.0, 1.0]])

In [146]:
histology = "../react-ui/src/assets/P57-16_updated/histology"


In [147]:
folder = os.path.join(histology,"01")
slice_folder = os.path.join(folder,"slices_HE")
if os.path.exists(slice_folder):
    print(slice_folder)
else:
    print("the folder does not exist")
images = [name for name in os.listdir(slice_folder) if os.path.isfile(os.path.join(slice_folder, name))]
print(len(images))
image = None
if len(images) > 0:
    image = images[0]

../react-ui/src/assets/P57-16_updated/histology/01/slices_HE
17


In [148]:
if not image == None:
    fullPath = os.path.join(slice_folder, image)
    readImage = matlib.imread(fullPath)
    print(readImage.shape)

(851, 626, 3)


In [166]:
# testing the matrix multiplications manually
# first I replicate what I have in javascript
# then below I replicate adrias example pdf
# the results show that the maths is all correct
# so theres no mistakes in adrias pdf or my javascript

In [ ]:
test_js_vector = np.asarray([77, 330, 177, 1])

In [163]:
test_js_matrix = np.asarray( [[-0.6539785013261993, -1.2529864276694398, -3.765398774658713, 1256.103850878735],
                           [3.9318829633011614, -0.47062047729014717, -0.5558042211829891, 284.41941706211304], 
                           [0.07086043352760864, 1.0005244597366796, -0.37536195122105953, -245.8121326422468], 
                           [0.0, 0.0, 0.0, 1.0]]
                        )

In [164]:
test_js_result = test_js_matrix.dot(test_js_vector)

In [165]:
#result is the same as what I get in javascript
print(test_js_result)

[125.78640203 333.49230058  23.37812709   1.        ]


In [169]:
# replicating the example in adrias pdf

In [170]:
test_adria_vector = np.asarray([77, 330, 175, 1])

In [171]:
test_adria_matrix = np.asarray( [[3.92, -0.54, -0.54, 293.53,],
                                 [-0.66, -1.26, -3.76, 1252.61],
                                 [0.097, 1.1, -0.4, -276.63],
                                 [0.0, 0.0, 0.0, 1.0]]
                        )

In [172]:
test_adria_result = test_adria_matrix.dot(test_adria_vector)

In [173]:
#result is the same as what adria has in his example pdf
print(test_adria_result)

[322.67  127.99   23.839   1.   ]
